In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

#### Test Audios

In [ ]:
test_audios=['Test_1.wav','Test_2.wav','Test_3.wav','Test_4.wav']

#Comparison for STT

In [ ]:
from transformers import pipeline, WhisperProcessor

# Load the Whisper ASR pipeline
base_model_id = "openai/whisper-large-v2"
base_stt_pipe = pipeline("automatic-speech-recognition", model=base_model_id, torch_dtype="auto", device=0)
processor = WhisperProcessor.from_pretrained(base_model_id)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="uz", task="transcribe")

Device set to use cuda:0


###Base STT model result (whisper-large-v2)

In [ ]:
for audio in test_audios:

  result=base_stt_pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids})
  print(f"Audio 1: {result['text']}")

Audio 1:  Qoş kentda Şafkat Mərzəyəev, Rossiya prezidentina kütüb aldı.
Audio 1:  Kundan kunga axola yomolla shar eda.
Audio 1:  Bugun məktabımızda kətti tadbır boldi.
Audio 1:  Hazirgi gündə suni intellekt boychi koplab layoxalar qilib keliyab man.


In [ ]:
import torch
from transformers import AutomaticSpeechRecognitionPipeline
from transformers import WhisperTokenizer,WhisperForConditionalGeneration,WhisperProcessor
from peft import PeftModel, PeftConfig

stt_model_id = "ShakhzoDavronov/whisper-large-lora-uz"
language = "Uzbek"
task = "transcribe"
stt_config = PeftConfig.from_pretrained(stt_model_id)
stt_model = WhisperForConditionalGeneration.from_pretrained(
    stt_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)

stt_model = PeftModel.from_pretrained(stt_model, stt_model_id)
stt_tokenizer = WhisperTokenizer.from_pretrained(stt_config.base_model_name_or_path, language=language, task=task)
stt_processor = WhisperProcessor.from_pretrained(stt_config.base_model_name_or_path, language=language, task=task)
stt_feature_extractor = stt_processor.feature_extractor
forced_decoder_ids = stt_processor.get_decoder_prompt_ids(language=language, task=task)
stt_pipe = AutomaticSpeechRecognitionPipeline(model=stt_model, tokenizer=stt_tokenizer, feature_extractor=stt_feature_extractor)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = stt_pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text

adapter_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

Device set to use cuda:0


###Fine-Tuned STT model result (whisper-large-lora-uz)

In [ ]:
print("Fine-Tuned STT model result:")
for audio in test_audios:
  text=transcribe(audio)
  print(text)

Fine-Tuned STT model result:
Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi
Kundan-kunga ahvola yomonlashar edi.
Bugun maktabimizda katta tadbir bo‘ldi
Hozirgi kunda suniy intellekt bo‘yicha ko‘plab loyihalar qilib kelyapman.


#Comparsion for NER

###Base NER model result (xlm-roberta-large)

In [1]:
from transformers import pipeline
base_ner_model_id='FacebookAI/xlm-roberta-large'
base_ner_pipe=pipeline('ner',model=base_ner_model_id,tokenizer=base_ner_model_id)

text = "Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi"
print("Base NER model Result:\n")
entities = base_ner_pipe(text)
for entity in entities:
    print(entity)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Device set to use cpu


Base NER model Result:

{'entity': 'LABEL_1', 'score': 0.5320926, 'index': 1, 'word': '▁Toshkent', 'start': 0, 'end': 8}
{'entity': 'LABEL_1', 'score': 0.61983335, 'index': 2, 'word': 'da', 'start': 8, 'end': 10}
{'entity': 'LABEL_1', 'score': 0.62647164, 'index': 3, 'word': '▁Shavkat', 'start': 11, 'end': 18}
{'entity': 'LABEL_1', 'score': 0.5044482, 'index': 4, 'word': '▁Mirziyoyev', 'start': 19, 'end': 29}
{'entity': 'LABEL_1', 'score': 0.5401966, 'index': 5, 'word': '▁Rossiya', 'start': 30, 'end': 37}
{'entity': 'LABEL_0', 'score': 0.5045715, 'index': 6, 'word': '▁prezidenti', 'start': 38, 'end': 48}
{'entity': 'LABEL_1', 'score': 0.6007423, 'index': 7, 'word': 'ni', 'start': 48, 'end': 50}
{'entity': 'LABEL_1', 'score': 0.63497436, 'index': 8, 'word': '▁ku', 'start': 51, 'end': 53}
{'entity': 'LABEL_1', 'score': 0.6514023, 'index': 9, 'word': 'tib', 'start': 53, 'end': 56}
{'entity': 'LABEL_1', 'score': 0.5837611, 'index': 10, 'word': '▁oldi', 'start': 57, 'end': 61}


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import pipeline

label_names=['O','B-PERSON','B-GPE','B-ORG','B-LOC','B-DATE','B-EVENT']
num_labels=len(label_names)
id_to_label = {i: label for i, label in enumerate(label_names)}
label_to_id = {label:i for i, label in enumerate(label_names)}

ner_model_id = "ShakhzoDavronov/xlm-roberta-lora-ner-uz"
ner_config = PeftConfig.from_pretrained(ner_model_id)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_config.base_model_name_or_path, num_labels=len(label_names),
                                                        id2label=id_to_label, label2id=label_to_id)

ner_tokenizer = AutoTokenizer.from_pretrained(ner_config.base_model_name_or_path)
ner_model = PeftModel.from_pretrained(ner_model,ner_model_id)

ner_pipe=pipeline('ner',model=ner_model,tokenizer=ner_tokenizer)

adapter_config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for ner. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DiffLlamaForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GemmaForTokenClassification', 'Gemma2ForTokenClassification', 'GlmForTokenClassification', 'GPT2ForTokenClassification', 'GPT2ForTokenClassification', 'GPTBigCo

###Fine-Tuned NER model result (xlm-roberta-lora-ner-uz)

In [ ]:
print("Fine-Tuned NER model Result:\n")
text="Toshkentda Shavkat Mirziyoyev Rossiya prezidentini kutib oldi"
ner=ner_pipe(text)
for entity in ner:
    print(entity)

Fine-Tuned NER model Result:

{'entity': 'B-LOC', 'score': 0.60946953, 'index': 1, 'word': '▁Toshkent', 'start': 0, 'end': 8}
{'entity': 'B-LOC', 'score': 0.55572474, 'index': 2, 'word': 'da', 'start': 8, 'end': 10}
{'entity': 'B-PERSON', 'score': 0.9300122, 'index': 3, 'word': '▁Shavkat', 'start': 11, 'end': 18}
{'entity': 'B-PERSON', 'score': 0.9212318, 'index': 4, 'word': '▁Mirziyoyev', 'start': 19, 'end': 29}
{'entity': 'B-GPE', 'score': 0.62043226, 'index': 5, 'word': '▁Rossiya', 'start': 30, 'end': 37}
